### Implementing FunkSVD

In this notebook we will take a look at writing our own function that performs FunkSVD, which will follow the steps you saw in the previous video.  If you find that you aren't ready to tackle this task on your own, feel free to skip to the following video where you can watch as I walk through the steps.

To test our algorithm, we will run it on the subset of the data you worked with earlier.  Run the cell below to get started.

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy import sparse
import svd_tests as t
%matplotlib inline

# Read in the datasets
movies = pd.read_csv('data/movies_clean.csv')
reviews = pd.read_csv('data/reviews_clean.csv')

del movies['Unnamed: 0']
del reviews['Unnamed: 0']

# Create user-by-item matrix
user_items = reviews[['user_id', 'movie_id', 'rating', 'timestamp']]
user_by_movie = user_items.groupby(['user_id', 'movie_id'])['rating'].max().unstack()

# Create data subset
user_movie_subset = user_by_movie[[73486, 75314,  68646, 99685]].dropna(axis=0)

[[ 10.  10.  10.  10.]
 [ 10.   4.   9.  10.]
 [  8.   9.  10.   5.]
 [  9.   8.  10.  10.]
 [ 10.   5.   9.   9.]
 [  6.   4.  10.   6.]
 [  9.   8.  10.   9.]
 [ 10.   5.   9.   8.]
 [  7.   8.  10.   8.]
 [  9.   5.   9.   7.]
 [  9.   8.  10.   8.]
 [  9.  10.  10.   9.]
 [ 10.   9.  10.   8.]
 [  5.   8.   5.   8.]
 [ 10.   8.  10.  10.]
 [  9.   9.  10.  10.]
 [  9.   8.   8.   8.]
 [ 10.   8.   1.  10.]
 [  5.   6.  10.  10.]
 [  8.   7.  10.   7.]]


In [13]:
ratings_mat = np.matrix(user_movie_subset)
print(ratings_mat)

[[ 10.  10.  10.  10.]
 [ 10.   4.   9.  10.]
 [  8.   9.  10.   5.]
 [  9.   8.  10.  10.]
 [ 10.   5.   9.   9.]
 [  6.   4.  10.   6.]
 [  9.   8.  10.   9.]
 [ 10.   5.   9.   8.]
 [  7.   8.  10.   8.]
 [  9.   5.   9.   7.]
 [  9.   8.  10.   8.]
 [  9.  10.  10.   9.]
 [ 10.   9.  10.   8.]
 [  5.   8.   5.   8.]
 [ 10.   8.  10.  10.]
 [  9.   9.  10.  10.]
 [  9.   8.   8.   8.]
 [ 10.   8.   1.  10.]
 [  5.   6.  10.  10.]
 [  8.   7.  10.   7.]]


In [14]:
print(user_movie_subset)

movie_id  73486  75314  68646  99685
user_id                             
265        10.0   10.0   10.0   10.0
1023       10.0    4.0    9.0   10.0
1683        8.0    9.0   10.0    5.0
6571        9.0    8.0   10.0   10.0
11639      10.0    5.0    9.0    9.0
13006       6.0    4.0   10.0    6.0
14076       9.0    8.0   10.0    9.0
14725      10.0    5.0    9.0    8.0
23548       7.0    8.0   10.0    8.0
24760       9.0    5.0    9.0    7.0
28713       9.0    8.0   10.0    8.0
30685       9.0   10.0   10.0    9.0
34110      10.0    9.0   10.0    8.0
34430       5.0    8.0    5.0    8.0
35150      10.0    8.0   10.0   10.0
43294       9.0    9.0   10.0   10.0
46849       9.0    8.0    8.0    8.0
50556      10.0    8.0    1.0   10.0
51382       5.0    6.0   10.0   10.0
51410       8.0    7.0   10.0    7.0


`1.` You will use the **user_movie_subset** matrix to show that your FunkSVD algorithm will converge.  In the below cell, use the comments and document string to assist you as you complete writing your own function to complete FunkSVD.  You may also want to try to complete the function on your own without the assistance of comments.  You may feel free to remove and add to the function in any way that gets you a working solution! 

**Notice:** There isn't a sigma matrix in this version of the matrix factorization.

In [15]:
def FunkSVD(ratings_mat, latent_features=4, learning_rate=0.0001, iters=100):
    '''
    This function performs matrix factorization using a basic form of FunkSVD with no regularization
    
    INPUT:
    ratings_mat - (numpy array) a matrix with users as rows, movies as columns, and ratings as values
    latent_features - (int) the number of latent features used
    learning_rate - (float) the learning rate 
    iters - (int) the number of iterations
    
    OUTPUT:
    user_mat - (numpy array) a user by latent feature matrix
    movie_mat - (numpy array) a latent feature by movie matrix
    '''
    
    # Set up useful values to be used through the rest of the function
    n_users = ratings_mat.shape[0] # number of rows in the matrix
    n_movies = ratings_mat.shape[1] # number of movies in the matrix
    num_ratings = np.count_nonzero(~np.isnan(ratings_mat)) # total number of ratings in the matrix
    
    # initialize the user and movie matrices with random values
    # helpful link: https://docs.scipy.org/doc/numpy/reference/generated/numpy.random.rand.html
    user_mat = np.random.rand(n_users, latent_features) # user matrix filled with random values of shape user x latent 
    movie_mat = np.random.rand(latent_features, n_movies) # movie matrix filled with random values of shape latent x movies
    
    # initialize sse at 0 for first iteration
    sse_accum = 0
    
    # header for running results
    print("Optimization Statistics")
    print("Iterations | Mean Squared Error ")
    
    # for each iteration
    for iteration in range(iters):

        # update our sse
        old_sse = sse_accum
        sse_accum = 0
        
        # For each user-movie pair
        for i in range(n_users):
            for j in range(n_movies):
                
                # if the rating exists
                if ratings_mat[i, j] > 0:
                    
                    # compute the error as the actual minus the dot product of the user and movie latent features
                    diff = ratings_mat[i, j] - np.dot(user_mat[i, :], movie_mat[:, j])
                    
                    # Keep track of the total sum of squared errors for the matrix
                    sse_accum += diff**2
                    
                    # update the values in each matrix in the direction of the gradient
                    for k in range(latent_features):
                        user_mat[i, k] += learning_rate * (2*diff) * movie_mat[k, j]
                        movie_mat[k, j] += learning_rate * (2*diff) * user_mat[i, k]

        # print results for iteration
        print("%d \t\t %f" % (iteration+1, sse_accum / num_ratings))
        
    return user_mat, movie_mat 

`2.` Try out your function on the **user_movie_subset** dataset.  First try 4 latent features, a learning rate of 0.005, and 10 iterations.  When you take the dot product of the resulting U and V matrices, how does the resulting **user_movie** matrix compare to the original subset of the data?

In [16]:
user_mat, movie_mat = FunkSVD(ratings_mat, latent_features = 4, 
                              learning_rate = 0.005, iters = 10)
# use your function with 4 latent features, lr of 0.005 and 10 iterations

Optimization Statistics
Iterations | Mean Squared Error 
1 		 43.290871
2 		 14.581519
3 		 4.029140
4 		 2.842991
5 		 2.726309
6 		 2.685807
7 		 2.656795
8 		 2.629090
9 		 2.600204
10 		 2.569394


In [17]:
#Compare the predicted and actual results
print(np.dot(user_mat, movie_mat))
print(ratings_mat)

[[  9.85482932   9.01653385  10.61136981  10.19819697]
 [  8.84740533   7.42863726   9.00873965   8.7296004 ]
 [  7.76404973   6.90706586   8.13841565   7.93825255]
 [  9.57098953   8.35442274  10.0019375    9.55931037]
 [  8.8595293    7.28457901   8.86522154   8.53182908]
 [  6.88416742   6.02911247   7.11826378   6.90149377]
 [  9.32032047   8.00345589   9.69513023   9.14360109]
 [  8.6255805    6.92448482   8.43818145   8.24535788]
 [  7.97515299   7.74424687   9.12914235   8.41236107]
 [  8.09464661   6.45658868   7.98609645   7.65334256]
 [  8.47285967   8.00204079   9.43576255   8.86379785]
 [  9.18530999   8.63539968  10.1921192    9.56422728]
 [  9.66702175   7.89602913   9.5945875    9.27795677]
 [  6.58021637   5.83474304   6.87109913   6.70234653]
 [  9.77927331   8.49497549  10.09618433   9.84826213]
 [  9.69778599   8.57299993  10.28909281   9.69465448]
 [  8.03940839   7.44461305   8.75119132   8.28787401]
 [  7.56677709   5.90207887   7.24578358   7.17231354]
 [  8.2068

**Write your summary here.**
The predicted ratings from the dot product are already starting to look a lot like the original data values even after only 10 iterations. You can see some extreme low values that are not captured well yet. The 5 in the second to last row in the first column is predicted as an 8, and the 4 in the second row and second column is predicted to be a 7. Clearly the model is not done learning, but things are looking good.

`3.` Let's try out the function again on the **user_movie_subset** dataset.  This time we will again use 4 latent features and a learning rate of 0.005.  However, let's bump up the number of iterations to 250.  When you take the dot product of the resulting U and V matrices, how does the resulting **user_movie** matrix compare to the original subset of the data?  What do you notice about your error at the end of the 250 iterations?

In [18]:
user_mat, movie_mat = FunkSVD(ratings_mat, latent_features = 4, 
                              learning_rate = 0.005, iters = 250)
#use your function with 4 latent features, lr of 0.005 and 250 iterations

Optimization Statistics
Iterations | Mean Squared Error 
1 		 39.850720
2 		 11.905130
3 		 3.539766
4 		 2.734019
5 		 2.641216
6 		 2.603607
7 		 2.573975
8 		 2.542955
9 		 2.507956
10 		 2.467960
11 		 2.422269
12 		 2.370248
13 		 2.311301
14 		 2.244921
15 		 2.170748
16 		 2.088644
17 		 1.998771
18 		 1.901665
19 		 1.798296
20 		 1.690083
21 		 1.578868
22 		 1.466826
23 		 1.356308
24 		 1.249647
25 		 1.148948
26 		 1.055901
27 		 0.971663
28 		 0.896817
29 		 0.831403
30 		 0.775025
31 		 0.726973
32 		 0.686355
33 		 0.652211
34 		 0.623591
35 		 0.599612
36 		 0.579485
37 		 0.562523
38 		 0.548141
39 		 0.535846
40 		 0.525225
41 		 0.515932
42 		 0.507681
43 		 0.500229
44 		 0.493373
45 		 0.486941
46 		 0.480784
47 		 0.474774
48 		 0.468797
49 		 0.462754
50 		 0.456554
51 		 0.450116
52 		 0.443365
53 		 0.436233
54 		 0.428655
55 		 0.420576
56 		 0.411943
57 		 0.402712
58 		 0.392845
59 		 0.382312
60 		 0.371095
61 		 0.359186
62 		 0.346589
63 		 0.333323
64 		

In [19]:
#Compare the predicted and actual results
print(np.dot(user_mat, movie_mat))
print(ratings_mat)

[[  9.99997396   9.99998948   9.9999957   10.00002368]
 [ 10.0000047    4.00000213   9.00000103   9.99999588]
 [  8.00000679   9.00000317  10.00000149   4.99999386]
 [  8.9999822    7.99999311   9.99999731  10.00001621]
 [  9.99998816   4.99999555   8.99999838   9.00001092]
 [  5.99998836   3.99999562   9.99999842   6.00001076]
 [  8.99999508   7.99999881   9.99999993   9.00000459]
 [ 10.00002485   5.00001068   9.00000473   7.99997758]
 [  7.00002469   8.00001043  10.00000452   7.99997762]
 [  8.99999794   4.99999893   8.99999955   7.00000205]
 [  9.00000992   8.00000403  10.00000168   7.99999104]
 [  9.00000927  10.00000369  10.00000149   8.99999158]
 [  9.99997126   8.99998815   9.99999498   8.00002611]
 [  5.00001455   8.00000635   5.00000275   7.99998665]
 [ 10.0000192    8.00000786  10.00000325   9.99998256]
 [  9.00000659   9.00000238  10.00000084   9.999994  ]
 [  9.00000472   8.0000015    8.00000049   7.99999576]
 [  9.99999639   7.99999838   0.9999992   10.00000313]
 [  4.9999

**Write your summary here.**
In this case, we were able to completely reconstruct the item-movie matrix to obtain an essentially 0 mean squared error. I obtained 0 MSE on iteration 181.

The last time we placed an **np.nan** value into this matrix the entire svd algorithm in python broke.  Let's see if that is still the case using your FunkSVD function.  In the below cell, I have placed a nan into the first cell of your numpy array.  

`4.` Use 4 latent features, a learning rate of 0.005, and 250 iterations.  Are you able to run your SVD without it breaking (something that was not true about the python built in)?  Do you get a prediction for the nan value?  What is your prediction for the missing value? Use the cells below to answer these questions.

In [20]:
# Here we are placing a nan into our original subset matrix
ratings_mat[0, 0] = np.nan
ratings_mat

matrix([[ nan,  10.,  10.,  10.],
        [ 10.,   4.,   9.,  10.],
        [  8.,   9.,  10.,   5.],
        [  9.,   8.,  10.,  10.],
        [ 10.,   5.,   9.,   9.],
        [  6.,   4.,  10.,   6.],
        [  9.,   8.,  10.,   9.],
        [ 10.,   5.,   9.,   8.],
        [  7.,   8.,  10.,   8.],
        [  9.,   5.,   9.,   7.],
        [  9.,   8.,  10.,   8.],
        [  9.,  10.,  10.,   9.],
        [ 10.,   9.,  10.,   8.],
        [  5.,   8.,   5.,   8.],
        [ 10.,   8.,  10.,  10.],
        [  9.,   9.,  10.,  10.],
        [  9.,   8.,   8.,   8.],
        [ 10.,   8.,   1.,  10.],
        [  5.,   6.,  10.,  10.],
        [  8.,   7.,  10.,   7.]])

In [21]:
# run SVD on the matrix with the missing value
user_mat, movie_mat = FunkSVD(ratings_mat, latent_features = 4, 
                              learning_rate = 0.005, iters = 250)
#use your function with 4 latent features, lr of 0.005 and 250 iterations

Optimization Statistics
Iterations | Mean Squared Error 
1 		 42.771678
2 		 15.191329
3 		 4.395352
4 		 2.905621
5 		 2.704591
6 		 2.633697
7 		 2.583629
8 		 2.535150
9 		 2.483215
10 		 2.426000
11 		 2.362510
12 		 2.292039
13 		 2.214084
14 		 2.128363
15 		 2.034862
16 		 1.933888
17 		 1.826107
18 		 1.712571
19 		 1.594700
20 		 1.474240
21 		 1.353172
22 		 1.233586
23 		 1.117530
24 		 1.006860
25 		 0.903102
26 		 0.807355
27 		 0.720248
28 		 0.641954
29 		 0.572246
30 		 0.510595
31 		 0.456273
32 		 0.408452
33 		 0.366286
34 		 0.328979
35 		 0.295813
36 		 0.266175
37 		 0.239554
38 		 0.215536
39 		 0.193790
40 		 0.174055
41 		 0.156125
42 		 0.139834
43 		 0.125043
44 		 0.111635
45 		 0.099507
46 		 0.088563
47 		 0.078715
48 		 0.069877
49 		 0.061966
50 		 0.054903
51 		 0.048613
52 		 0.043023
53 		 0.038063
54 		 0.033670
55 		 0.029784
56 		 0.026350
57 		 0.023318
58 		 0.020641
59 		 0.018279
60 		 0.016195
61 		 0.014356
62 		 0.012733
63 		 0.011300
64 		

In [22]:
# Run this cell to see if you were able to predict for the missing value
preds = np.dot(user_mat, movie_mat)
print("The predicted value for the missing rating is {}:".format(preds[0,0]))
print()
print("The actual value for the missing rating is {}:".format(ratings_mat[0,0]))
print()
assert np.isnan(preds[0,0]) == False
print("That's right! You just predicted a rating for a user-movie pair that was never rated!")
print("But if you look in the original matrix, this was actually a value of 10. Not bad!")

The predicted value for the missing rating is 9.02784478093777:

The actual value for the missing rating is nan:

That's right! You just predicted a rating for a user-movie pair that was never rated!
But if you look in the original matrix, this was actually a value of 10. Not bad!


In [23]:
preds

array([[  9.02784478,  10.00000066,  10.00000037,   9.99999922],
       [  9.99999833,   3.99999933,   8.99999959,  10.00000162],
       [  7.99999667,   8.99999858,   9.99999914,   5.00000334],
       [  8.99999349,   7.99999738,   9.99999844,  10.00000645],
       [  9.99999847,   4.9999995 ,   8.99999976,   9.00000154],
       [  6.00000019,   4.00000018,  10.00000016,   5.99999986],
       [  9.00000371,   8.00000158,  10.00000101,   8.99999646],
       [  9.99999808,   4.99999932,   8.99999962,   8.00000191],
       [  7.00000094,   8.00000039,  10.00000027,   7.99999918],
       [  9.00000626,   5.00000259,   9.00000159,   6.99999391],
       [  8.99999273,   7.99999701,   9.99999822,   8.00000721],
       [  8.99999368,   9.99999748,   9.99999855,   9.00000633],
       [  9.99999922,   8.99999982,   9.99999999,   8.0000009 ],
       [  5.00000364,   8.00000152,   5.00000098,   7.99999657],
       [  9.99999832,   7.99999941,   9.99999972,  10.00000177],
       [  9.00000274,   9

Now let's extend this to a more realistic example. Unfortunately, running this function on your entire user-movie matrix is still not something you likely want to do on your local machine.  However, we can see how well this example extends to 1000 users.  In the above portion, you were using a very small subset of data with no missing values.

`5.` Given the size of this matrix, this will take quite a bit of time.  Consider the following hyperparameters: 4 latent features, 0.005 learning rate, and 20 iterations.  Grab a snack, take a walk, and this should be done running in a bit.

In [24]:
# Setting up a matrix of the first 1000 users with movie ratings
first_1000_users = np.matrix(user_by_movie.head(1000))

# perform funkSVD on the matrix of the top 1000 users
user_mat, movie_mat = FunkSVD(first_1000_users, latent_features = 4, 
                              learning_rate = 0.005, iters = 20)
#fit to 1000 users with 4 latent features, lr of 0.005, and 20 iterations

Optimization Statistics
Iterations | Mean Squared Error 
1 		 23.338404
2 		 10.799496
3 		 7.379278
4 		 5.676927
5 		 4.608089
6 		 3.861149
7 		 3.305116
8 		 2.874323
9 		 2.531648
10 		 2.253973
11 		 2.025784
12 		 1.836116
13 		 1.676960
14 		 1.542317
15 		 1.427601
16 		 1.329225
17 		 1.244335
18 		 1.170624
19 		 1.106215
20 		 1.049577


`6.` Now that you have a set of predictions for each user-movie pair,  let's answer a few questions about your results. Provide the correct values for each of the variables below, and check your solutions using the tests below.

In [25]:
# Replace each of the comments below with the correct values
# How many actual ratings exist in first_1000_users
num_ratings = np.count_nonzero(~np.isnan(first_1000_users))
print("The number of actual ratings in the first_1000_users is {}.".format(num_ratings))
print()

# How many ratings did we make for user-movie pairs that didn't actually have ratings
ratings_for_missing = first_1000_users.shape[0]*first_1000_users.shape[1] - num_ratings
print("The number of ratings made for user-movie pairs that didn't have ratings is {}".format(ratings_for_missing))


The number of actual ratings in the first_1000_users is 10852.

The number of ratings made for user-movie pairs that didn't have ratings is 31234148


In [26]:
# Test your results against the solution
assert num_ratings == 10852, "Oops!  The number of actual ratings doesn't quite look right."
assert ratings_for_missing == 31234148, "Oops!  The number of movie-user pairs that you made ratings for that didn't actually have ratings doesn't look right."

# Make sure you made predictions on all the missing user-movie pairs
preds = np.dot(user_mat, movie_mat)
assert np.isnan(preds).sum() == 0
print("Nice job!  Looks like you have predictions made for all the missing user-movie pairs! But I still have one question... How good are they?")

Nice job!  Looks like you have predictions made for all the missing user-movie pairs! But I still have one question... How good are they?
